In [1]:
!pip install transformers trl peft accelerate datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Steps

1. Unsupervised (Pre-training)
2. SFT (Supervised Fine-tuning)
3. RLHF
4. Inferencing

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from datasets import load_dataset, Dataset
from trl import SFTTrainer, DPOTrainer, DPOConfig
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model # prepare_model_for_kbit_training - to convert the model into 4 bit, 16 it in
from accelerate import Accelerator
import torch

## 2. SFT (Supervised Finetuning)

In [4]:
## SFT configuration
sft_config={
    "model_ckpt": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "torch_dtype": torch.float16,
            "trust_remote_code": True,

            "use_lora": True,
            "r": 16,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],

            "output_dir": "sft-tiny-chatbot",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            #"train_cln_name": "text",
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5
}

In [11]:
class TrainSFT:
  def __init__(self,data,config) -> None:
    self.data=data
    self.config=config

  def prepare_lora_model(self):
    self.lora_config=LoraConfig(r=self.config["r"],
      lora_alpha=self.config["lora_alpha"],
      lora_dropout=self.config["lora_dropout"],
      bias=self.config["bias"],
      task_type=self.config["task_type"],
      target_modules=self.config["target_modules"])

    self.model=get_peft_model(self.model,self.lora_config)

  def load_model_tokenizer(self):
    self.model=AutoModelForCausalLM.from_pretrained(
      self.config["model_ckpt"],
      load_in_4bit=self.config["load_in_4bit"],
      device_map=self.config["device_map"],
      torch_dtype=self.config["torch_dtype"])
    self.model.config.use_cache=False
    self.model.config.pretraining_tp=1
    self.model = prepare_model_for_kbit_training(self.model)

    if self.config["use_lora"]:
      self.prepare_lora_model()

    self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
    self.tokenizer.pad_token = self.tokenizer.eos_token

  def set_training_args(self):
      return TrainingArguments(
        output_dir=self.config["output_dir"],
        per_device_train_batch_size=self.config["per_device_train_batch_size"],
        gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
        optim=self.config["optim"],
        learning_rate=self.config["learning_rate"],
        lr_scheduler_type=self.config["lr_scheduler_type"],
        save_strategy=self.config["save_strategy"],
        logging_steps=self.config["logging_steps"],
        num_train_epochs=self.config["num_train_epochs"],
        max_steps=self.config["max_steps"],
        fp16=self.config["fp16"],
        push_to_hub=self.config["push_to_hub"],
        neftune_noise_alpha=self.config["neftune_noise_alpha"],
        report_to="none")

  def create_trainer(self):
    self.load_model_tokenizer()
    if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = SFTTrainer(
                model=self.model,
                train_dataset=self.data,
                peft_config=self.lora_config,
                args=self.set_training_args(),
                tokenizer=self.tokenizer)


  def train_and_save_model(self):
    self.create_trainer()
    self.trainer.train()
    self.trainer.save_model(self.config["output_dir"])
    self.tokenizer.save_pretrained(self.config["output_dir"])


In [6]:
def create_data():
  data=load_dataset("tatsu-lab/alpaca", split="train")
  data_df = data.to_pandas()
  data_df = data_df[:700]
  data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "Human: " + x["instruction"] + " " + x["input"] + " Assistant: "+ x["output"], axis=1)
  data = Dataset.from_pandas(data_df)
  return data

data=create_data()
data

README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 700
})

In [12]:
train_sft=TrainSFT(data,sft_config)

In [13]:
train_sft.train_and_save_model()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

Tokenizing train dataset:   0%|          | 0/700 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/700 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended,

Step,Training Loss
100,2.133700
200,1.738000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


## Preference Alignment - DPO
- Fist we Training or finetunining model using PEFT (SFT)
- Then will retain for preference alignment for controlled response using DPO

In [14]:
dpo_config = {
            "model_ckpt": "snshrivas10/sft-tiny-chatbot",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 8,
            "lora_alpha": 8,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "tiny-chatbot-dpo",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "neftune_noise_alpha": 5,
            "beta": 0.1,
            "loss_type": "kto_pair",
            "max_length": 768,
            "max_prompt_length": 512,
            "max_target_length": 256,
            "is_encoder_decoder": False
        }

DPO optimizes a policy (trainable model) by comparing its outputs against a reference model:


In [15]:
class TrainDPO:
  def __init__(self,data,config) -> None:
    self.data=data
    self.config=config

  def prepare_lora_model(self):
    self.lora_config = LoraConfig(
        r=self.config["r"],
        lora_alpha=self.config["lora_alpha"],
        lora_dropout=self.config["lora_dropout"],
        bias=self.config["bias"],
        task_type=self.config["task_type"],
        target_modules=self.config["target_modules"]
      )
    self.model = get_peft_model(self.model, self.lora_config) #THIS MODEL WHICH I AM GOING TO TRAIN FURTHER ON MY HUMAN PREFRENCE DATA
    self.model_ref = get_peft_model(self.model_ref, self.lora_config) #SAME MODEL IN THIS VARIABLE FOR THE REFRENCE


  def load_model_tokenizer(self):
    self.model = AutoModelForCausalLM.from_pretrained(
        self.config["model_ckpt"],
        load_in_4bit=self.config["load_in_4bit"],
        device_map=self.config["device_map"],
        torch_dtype=self.config["torch_dtype"])

    self.model_ref = AutoModelForCausalLM.from_pretrained(
          self.config["model_ckpt"],
          load_in_4bit=self.config["load_in_4bit"],
          device_map=self.config["device_map"],
          torch_dtype=self.config["torch_dtype"])

    self.model.config.use_cache=False
    self.model.config.pretraining_tp=1
    self.model = prepare_model_for_kbit_training(self.model)
    if self.config["use_lora"]:
        self.prepare_lora_model()

    self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
    self.tokenizer.pad_token = self.tokenizer.eos_token


  def set_training_args(self):
    return DPOConfig(
      output_dir=self.config["output_dir"],
      per_device_train_batch_size=self.config["per_device_train_batch_size"],
      gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
      optim=self.config["optim"],
      learning_rate=self.config["learning_rate"],
      lr_scheduler_type=self.config["lr_scheduler_type"],
      save_strategy=self.config["save_strategy"],
      logging_steps=self.config["logging_steps"],
      num_train_epochs=self.config["num_train_epochs"],
      max_steps=self.config["max_steps"],
      fp16=self.config["fp16"],
      push_to_hub=self.config["push_to_hub"],
      neftune_noise_alpha=self.config["neftune_noise_alpha"],
      report_to="none",
      remove_unused_columns=False)

  def create_trainer(self):
    self.load_model_tokenizer()

    if self.config["use_lora"]:
        print(self.model.print_trainable_parameters())
        self.trainer = DPOTrainer(
            self.model,
            self.model_ref,
            args=self.set_training_args(),
            train_dataset=self.data,
            tokenizer=self.tokenizer)


  def train_and_save_model(self):
    self.create_trainer()
    self.trainer.train()
    self.trainer.save_model(self.config["output_dir"])
    self.tokenizer.save_pretrained(self.config["output_dir"])

In [16]:
def create_data():
    df = load_dataset("Anthropic/hh-rlhf", split="train").to_pandas()
    df["prompt"] = df["chosen"].apply(lambda x: x.split("Assistant: ")[0])
    df["chosen"] = df["chosen"].apply(lambda x: "Assistant: "+ x.split("Assistant: ")[-1])
    df["rejected"] = df["rejected"].apply(lambda x: "Assistant: " + x.split("Assistant: ")[-1])
    df = df.sample(1000)
    data = Dataset.from_pandas(df)
    return data

data=create_data()
data

README.md:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

test.jsonl.gz:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

Dataset({
    features: ['chosen', 'rejected', 'prompt', '__index_level_0__'],
    num_rows: 1000
})

In [17]:
data[0]

{'chosen': 'Assistant: I’m sorry, I’m not sure what I could clarify here.  The Normandy Empire eventually became the Kingdom of England, which is what you see as England today.  That was in 1066.  Is there anything else I could clarify?',
 'rejected': 'Assistant: Which question was that?  I’m not sure what else you might be asking, so I’m hoping you’ll let me know.',
 'prompt': '\n\nHuman: At what point in history did Christianity replace all other religions in Western Europe?\n\n',
 '__index_level_0__': 59219}

In [18]:
data[0]["chosen"]

'Assistant: I’m sorry, I’m not sure what I could clarify here.  The Normandy Empire eventually became the Kingdom of England, which is what you see as England today.  That was in 1066.  Is there anything else I could clarify?'

In [19]:
data[0]["rejected"]

'Assistant: Which question was that?  I’m not sure what else you might be asking, so I’m hoping you’ll let me know.'

In [20]:
data[0]["prompt"]

'\n\nHuman: At what point in history did Christianity replace all other religions in Western Europe?\n\n'

In [21]:
train_dpo=TrainDPO(data,dpo_config)

In [22]:
train_dpo.train_and_save_model()

adapter_config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


adapter_model.safetensors:   0%|          | 0.00/9.02M [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

...

Applying chat template to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Step,Training Loss
100,0.695500
200,0.703800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


## Inferencing

#### load the model from huggingface

#### fit into the transformer pipeline

#### then make a prediction out of it